This notebook is a benchmark query comparison with the Postgres queries

In [9]:
%%configure -f
{"name": "brian-query-benchmarks-2", "executorMemory": "36G", "numExecutors": 15, "executorCores": 10,
 "conf": {"spark.yarn.appMasterEnv.PYSPARK_PYTHON":"python3"}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
13,application_1602298274728_0014,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
12,application_1602298274728_0013,pyspark,busy,Link,Link,
13,application_1602298274728_0014,pyspark,idle,Link,Link,✔


In [10]:
import numpy as np
import matplotlib.pyplot as plt
import time

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
class Timer(object):
    def __enter__(self):
        self.start_time = time.perf_counter()
        return self
   
    def __exit__(self, *exc_info):
        self.end_time = time.perf_counter()
        elapsed = self.end_time-self.start_time
        print('Elapsed time [sec]:', elapsed)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# creates a temporary "database" table
with Timer():
    drpall = spark.read.parquet('hdfs:///manga/brian-test/dr15/v2_4_3/drpall')
    drpall.createOrReplaceTempView('drpall')
    dapall = spark.read.parquet('hdfs:///manga/brian-test/dr15/v2_4_3/dapall')
    dapall.createOrReplaceTempView('dapall')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Elapsed time [sec]: 6.055798895002226

In [13]:
# creates a temporary "database" table for maps
with Timer():
    maps = spark.read.parquet('hdfs:///manga/brian-test/dr15/v2_4_3/maps')
    maps.createOrReplaceTempView('maps')
    
# data.getNumPartitions - number of tasks need to run in total
# numExecutor * numCores can run at any given time

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Elapsed time [sec]: 1.4605491300026188

In [14]:
# load the DRP cubes
with Timer():
    cubes = spark.read.parquet('hdfs:///manga/arik-test/dr15/v2_4_3/logcube_voxel')
    cubes.createOrReplaceTempView('cubes')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Elapsed time [sec]: 1.3701034010009607

## Query 1
Select all galaxies with an H-alpha flux value (in any spaxel) > 25 

### Raw SQL 

SELECT mangadatadb.cube.mangaid AS "cube.mangaid", mangadatadb.cube.plate AS "cube.plate", concat(mangadatadb.cube.plate, '-', mangadatadb.ifudesign.name) AS "cube.plateifu", mangadatadb.ifudesign.name AS "ifu.name", mangadapdb.cleanspaxelprop7.emline_gflux_ha_6564 AS emline_gflux_ha_6564, mangadapdb.cleanspaxelprop7.x AS "spaxelprop.x", mangadapdb.cleanspaxelprop7.y AS "spaxelprop.y"
FROM mangadatadb.cube JOIN mangadatadb.ifudesign ON mangadatadb.ifudesign.pk = mangadatadb.cube.ifudesign_pk JOIN mangadapdb.file ON mangadatadb.cube.pk = mangadapdb.file.cube_pk JOIN mangadapdb.cleanspaxelprop7 ON mangadapdb.file.pk = mangadapdb.cleanspaxelprop7.file_pk JOIN mangadatadb.pipeline_info AS drpalias ON drpalias.pk = mangadatadb.cube.pipeline_info_pk JOIN mangadatadb.pipeline_info AS dapalias ON dapalias.pk = mangadapdb.file.pipeline_info_pk
WHERE mangadapdb.cleanspaxelprop7.emline_gflux_ha_6564 > 25.0 AND drpalias.pk = 32 AND dapalias.pk = 34;

### Postgres Results
Query 1 
(104832 rows) (multiple bintypes)

1st run
Time: 169874.996 ms (02:49.875)

2nd run - cached
Time: 568.400 ms

selected on bintype HYB10 only - 1st run

Via bintype.name - (52174 rows) Time: 375.401 ms

Via file.filename -   (52174 rows) Time: 2176.983 ms (00:02.177)


## Querying maps parquet table directly

In [7]:
spark.sql('''describe maps''').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|            plateifu|   string|   null|
|             mangaid|   string|   null|
|                  ra|   double|   null|
|                 dec|   double|   null|
|             daptype|   string|   null|
|          spaxel_idx|   bigint|   null|
|                   x|   bigint|   null|
|                   y|   bigint|   null|
| spx_skycoo_on_sky_x|   double|   null|
| spx_skycoo_on_sky_y|   double|   null|
|spx_ellcoo_ellipt...|   double|   null|
|     spx_ellcoo_r_re|   double|   null|
|spx_ellcoo_ellipt...|   double|   null|
|           spx_mflux|   double|   null|
|      spx_mflux_ivar|   double|   null|
|             spx_snr|   double|   null|
|binid_binned_spectra|   bigint|   null|
|binid_stellar_con...|   bigint|   null|
|binid_em_line_mom...|   bigint|   null|
|binid_em_line_models|   bigint|   null|
+--------------------+---------+-------+
only showing top

In [8]:
# straight count (322 ms with postgres on cleanspaxelprop7)
with Timer():
    spark.sql('''select plateifu, emline_gflux_ha_6564, x, y from maps where emline_gflux_ha_6564 > 25 and binid_binned_spectra != -1 and binid_stellar_continua != -1 and binid_em_line_models != -1 and binid_em_line_moments != -1 and binid_spectral_indices != -1''').count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

52174
Elapsed time [sec]: 5.9793291590031

In [9]:
# querying the parquet table itself (516 ms with postgres on cleanspaxelprop7)
with Timer():
    spark.sql('''select plateifu, emline_gflux_ha_6564, x, y from maps where emline_gflux_ha_6564 > 25 and binid_binned_spectra != -1 and binid_stellar_continua != -1 and binid_em_line_models != -1 and binid_em_line_moments != -1 and binid_spectral_indices != -1''').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+---+---+
| plateifu|emline_gflux_ha_6564|  x|  y|
+---------+--------------------+---+---+
|8551-3701|  26.905492782592773| 18| 20|
|8551-3701|  31.592317581176758| 19| 20|
|8551-3701|  33.098114013671875| 20| 20|
|8551-3701|  31.165016174316406| 21| 20|
|8551-3701|  27.630207061767578| 22| 20|
|8551-3701|   25.37577247619629| 17| 21|
|8551-3701|  33.406036376953125| 18| 21|
|8551-3701|   40.30801010131836| 19| 21|
|8551-3701|    42.7925910949707| 20| 21|
|8551-3701|   43.42845916748047| 21| 21|
|8551-3701|   39.72331237792969| 22| 21|
|8551-3701|  34.885719299316406| 23| 21|
|8551-3701|  30.181264877319336| 24| 21|
|8551-3701|  26.739524841308594| 25| 21|
|8551-3701|  27.167835235595703| 17| 22|
|8551-3701|  36.835113525390625| 18| 22|
|8551-3701|   41.56124496459961| 19| 22|
|8551-3701|  44.282310485839844| 20| 22|
|8551-3701|   45.60519790649414| 21| 22|
|8551-3701|  44.445220947265625| 22| 22|
+---------+--------------------+---+---+
only showing top

In [10]:
# collecting the results
with Timer():
    res = spark.sql('''select plateifu, emline_gflux_ha_6564, x, y from maps where emline_gflux_ha_6564 > 25 and binid_binned_spectra != -1 and binid_stellar_continua != -1 and binid_em_line_models != -1 and binid_em_line_moments != -1 and binid_spectral_indices != -1''').collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Elapsed time [sec]: 2.5561871440004325

In [11]:
res[0:10]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(plateifu='8551-3701', emline_gflux_ha_6564=26.905492782592773, x=18, y=20), Row(plateifu='8551-3701', emline_gflux_ha_6564=31.592317581176758, x=19, y=20), Row(plateifu='8551-3701', emline_gflux_ha_6564=33.098114013671875, x=20, y=20), Row(plateifu='8551-3701', emline_gflux_ha_6564=31.165016174316406, x=21, y=20), Row(plateifu='8551-3701', emline_gflux_ha_6564=27.630207061767578, x=22, y=20), Row(plateifu='8551-3701', emline_gflux_ha_6564=25.37577247619629, x=17, y=21), Row(plateifu='8551-3701', emline_gflux_ha_6564=33.406036376953125, x=18, y=21), Row(plateifu='8551-3701', emline_gflux_ha_6564=40.30801010131836, x=19, y=21), Row(plateifu='8551-3701', emline_gflux_ha_6564=42.7925910949707, x=20, y=21), Row(plateifu='8551-3701', emline_gflux_ha_6564=43.42845916748047, x=21, y=21)]

## Join cube and maps Parquet tables together
To mimic as much as possible the Postgres query

In [ ]:
spark.sql('''describe cubes''').show()

In [ ]:
# count 
with Timer():
    spark.sql('''select c.plateid, c.mangaid, c.ifudsgn, concat(c.plateid,'-',c.ifudsgn) as plateifu, c.ra, c.dec, m.x,m.y,m.emline_gflux_ha_6564 from cubes as c cross join maps as m on m.plateifu=plateifu where m.emline_gflux_ha_6564 > 25''').count()

In [ ]:
# show query with join between cube and maps
with Timer():
    spark.sql('''select c.plateid, c.mangaid, c.ifudsgn, concat(c.plateid,'-',c.ifudsgn) as plateifu, c.ra, c.dec, m.x,m.y,m.emline_gflux_ha_6564 from cubes as c cross join maps as m on m.plateifu=plateifu where m.emline_gflux_ha_6564 > 25 and m.binid_binned_spectra != -1 and m.binid_stellar_continua != -1 and m.binid_em_line_models != -1 and m.binid_em_line_moments != -1 and m.binid_spectral_indices != -1''').show()

In [ ]:
# run a collect on the query
with Timer():
    results = spark.sql('''select c.plateid, c.mangaid, c.ifudsgn, concat(c.plateid,'-',c.ifudsgn) as plateifu, c.ra, c.dec, m.x,m.y,m.emline_gflux_ha_6564 from cubes as c cross join maps as m on m.plateifu=plateifu where m.emline_gflux_ha_6564 > 25 and m.binid_binned_spectra != -1 and m.binid_stellar_continua != -1 and m.binid_em_line_models != -1 and m.binid_em_line_moments != -1 and m.binid_spectral_indices != -1''').cache()
    res = results.collect() 


In [ ]:
type(data)

In [ ]:
#.cache will cache sql results
#.createDataFrame pipe to Spark dataframe
#.toPands to create Pandas dataframe

## Query 2
Select all galaxies with an H-alpha flux value > 5 in more than 20% of their good spaxels

### Raw SQL 

SELECT anon_1.mangadatadb_cube_mangaid, anon_1.mangadatadb_cube_plate, concat(anon_1.mangadatadb_cube_plate, '-', anon_1.mangadatadb_ifudesign_name) AS plateifu, anon_1.mangadatadb_ifudesign_name
FROM (SELECT mangadatadb.cube.mangaid AS mangadatadb_cube_mangaid, mangadatadb.cube.plate AS mangadatadb_cube_plate, concat(mangadatadb.cube.plate, '-', mangadatadb.ifudesign.name) AS plateifu, mangadatadb.ifudesign.name AS mangadatadb_ifudesign_name, mangadapdb.cleanspaxelprop7.emline_gflux_ha_6564 AS mangadapdb_cleanspaxelprop7_emline_gflux_ha_6564, mangadapdb.cleanspaxelprop7.x AS mangadapdb_cleanspaxelprop7_x, mangadapdb.cleanspaxelprop7.y AS mangadapdb_cleanspaxelprop7_y
FROM mangadatadb.cube JOIN mangadatadb.ifudesign ON mangadatadb.ifudesign.pk = mangadatadb.cube.ifudesign_pk JOIN mangadapdb.file ON mangadatadb.cube.pk = mangadapdb.file.cube_pk JOIN mangadapdb.cleanspaxelprop7 ON mangadapdb.file.pk = mangadapdb.cleanspaxelprop7.file_pk JOIN mangadatadb.pipeline_info AS drpalias ON drpalias.pk = mangadatadb.cube.pipeline_info_pk JOIN mangadatadb.pipeline_info AS dapalias ON dapalias.pk = mangadapdb.file.pipeline_info_pk JOIN (SELECT mangadapdb.cleanspaxelprop7.file_pk AS binfile, count(mangadapdb.cleanspaxelprop7.pk) AS goodcount
FROM mangadapdb.cleanspaxelprop7
WHERE mangadapdb.cleanspaxelprop7.binid_binned_spectra != -1 AND mangadapdb.cleanspaxelprop7.binid_stellar_continua != -1 AND mangadapdb.cleanspaxelprop7.binid_spectral_indices != -1 AND mangadapdb.cleanspaxelprop7.binid_em_line_moments != -1 AND mangadapdb.cleanspaxelprop7.binid_em_line_models != -1 GROUP BY mangadapdb.cleanspaxelprop7.file_pk) AS bingood ON bingood.binfile = mangadapdb.cleanspaxelprop7.file_pk JOIN (SELECT mangadapdb.cleanspaxelprop7.file_pk AS valfile, count(mangadapdb.cleanspaxelprop7.pk) AS valcount
FROM mangadapdb.cleanspaxelprop7
WHERE mangadapdb.cleanspaxelprop7.emline_gflux_ha_6564 > 5 GROUP BY mangadapdb.cleanspaxelprop7.file_pk) AS goodhacount ON goodhacount.valfile = mangadapdb.cleanspaxelprop7.file_pk
WHERE drpalias.pk = 32 AND dapalias.pk = 34 AND goodhacount.valcount >= 0.2 * bingood.goodcount) AS anon_1 GROUP BY anon_1.mangadatadb_cube_mangaid, anon_1.mangadatadb_cube_plate, concat(anon_1.mangadatadb_cube_plate, '-', anon_1.mangadatadb_ifudesign_name), anon_1.mangadatadb_ifudesign_name

### Postgres Results
(671 rows)

1st run
Time: 1998737.520 ms (33:18.738)

2nd run
Time: 2071235.639 ms (34:31.236)


In [12]:
type(maps)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.dataframe.DataFrame'>

In [13]:
good_spaxels = ((maps.binid_binned_spectra != -1) & 
                   (maps.binid_stellar_continua != -1) & 
                   (maps.binid_em_line_models != -1) & 
                   (maps.binid_em_line_moments != -1) & 
                   (maps.binid_spectral_indices != -1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
with Timer():
    # get total counts of number of good spaxels, grouped by plateifu
    tc = maps.filter(good_spaxels).groupby('plateifu').count().withColumnRenamed('count', 'totalc')
    # get counts of number of good spaxels with H-alpha > 5, grouped by plateifu
    fc = maps.filter(good_spaxels).filter(maps['emline_gflux_ha_6564'] > 5).groupby('plateifu').count().withColumnRenamed('count', 'filterc')
    # join the tables and filter where 
    tmp = tc.join(fc, 'plateifu')
    tmp.filter(tmp.filterc >= 0.2 * tmp.totalc).count()
    
# 3-5 seconds, 16 seconds

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

664
Elapsed time [sec]: 6.995045989999198

In [15]:
# try bucket-ing on plateifu (bucketby - used for writing out - optimizing on-disk structure)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
tmp.filter(tmp.filterc >= 0.2 * tmp.totalc).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+-------+
|  plateifu|totalc|filterc|
+----------+------+-------+
| 8262-1901|   400|    120|
|8984-12705|  2803|    636|
| 8262-3702|   842|    354|
| 8333-1902|   437|    156|
| 8616-1901|   413|    340|
| 8952-6104|  1340|    909|
|9674-12703|  2630|    908|
| 7968-3701|   761|    259|
| 8261-1901|   450|    131|
| 8453-6102|   981|    289|
|9674-12701|  2577|   1418|
|8081-12704|  1314|    289|
| 8442-1901|   310|     89|
| 8482-3702|   836|    328|
|9675-12703|  2643|   2618|
| 8442-6102|   986|    462|
| 8615-1902|   386|    231|
| 7977-1902|   396|    317|
| 8247-3701|   861|    183|
| 8447-1901|   442|    279|
+----------+------+-------+
only showing top 20 rows

In [ ]:
# this cell takes forever to run on cube-voxels
from pyspark.sql.functions import concat, col, lit
with Timer():
    ctemp = cubes.withColumn('plateifu', concat(col("plateid"), lit("-"), col("ifudsgn")))
    ctemp.join(tmp, 'plateifu').select('plateifu', 'mangaid', 'totalc', 'filterc').count()

In [ ]:
ctemp = cubes.withColumn('plateifu', concat(col("plateid"), lit("-"), col("ifudsgn")))
cg = ctemp.groupby('plateifu').count().alias('counts')
cg.show(5)

In [ ]:
data_joined = ctemp.join(cg, 'plateifu').dropDuplicates().show(5)

In [ ]:
ctemp.show()

### Spark SQL

In [15]:
totsql = """ select f.plateifu, count(f.*) as tcount from maps as f \
where f.binid_binned_spectra != -1 and f.binid_stellar_continua != -1 and f.binid_em_line_models != -1 \
and f.binid_em_line_moments != -1 and f.binid_spectral_indices != -1 group by f.plateifu
"""
hasql = """ select f.plateifu, count(f.*) as vcount from maps as f \
where f.emline_gflux_ha_6564 > 5 and f.binid_binned_spectra != -1 and f.binid_stellar_continua != -1 \
and f.binid_em_line_models != -1 and f.binid_em_line_moments != -1 and f.binid_spectral_indices != -1 \
group by f.plateifu
"""
sql = """ select t.plateifu, t.tcount, v.vcount from (select f.plateifu, count(f.*) as tcount from maps as f \
where f.binid_binned_spectra != -1 and f.binid_stellar_continua != -1 and f.binid_em_line_models != -1 \
and f.binid_em_line_moments != -1 and f.binid_spectral_indices != -1 group by f.plateifu) as t, (select f.plateifu, count(f.*) as vcount from maps as f \
where f.emline_gflux_ha_6564 > 5 and f.binid_binned_spectra != -1 and f.binid_stellar_continua != -1 \
and f.binid_em_line_models != -1 and f.binid_em_line_moments != -1 and f.binid_spectral_indices != -1 \
group by f.plateifu) as v where t.plateifu=v.plateifu and \
v.vcount >= 0.2*t.tcount
"""

with Timer():
    spark.sql(sql).count()

# 4 numExec, 18 cores, 36 execMem
# show 156.64 seconds
# count 161.6 seconds [~2.7 min]

# 15 numExec, 10 cores, 36 execMem
# count 631.33

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

664
Elapsed time [sec]: 631.3303209980077

## Query 3
Select all galaxies with an NSA sersic_n index < 2, an H-alpha sumed-EW > 6, and an NSA sersic log stellar mass between 9.5-11

### Raw SQL 

SELECT mangadatadb.cube.mangaid AS "cube.mangaid", mangadatadb.cube.plate AS "cube.plate", concat(mangadatadb.cube.plate, '-', mangadatadb.ifudesign.name) AS "cube.plateifu", mangadatadb.ifudesign.name AS "ifu.name", mangadapdb.cleanspaxelprop7.emline_sew_ha_6564 AS emline_sew_ha_6564, mangasampledb.nsa.sersic_n AS "nsa.sersic_n", CAST(CASE WHEN (mangasampledb.nsa.sersic_mass > 0.0) THEN log(mangasampledb.nsa.sersic_mass) WHEN (mangasampledb.nsa.sersic_mass = 0.0) THEN 0.0 END AS FLOAT) AS "nsa.sersic_logmass", mangadapdb.cleanspaxelprop7.x AS "spaxelprop.x", mangadapdb.cleanspaxelprop7.y AS "spaxelprop.y"
FROM mangadatadb.cube JOIN mangadatadb.ifudesign ON mangadatadb.ifudesign.pk = mangadatadb.cube.ifudesign_pk JOIN mangadapdb.file ON mangadatadb.cube.pk = mangadapdb.file.cube_pk JOIN mangadapdb.cleanspaxelprop7 ON mangadapdb.file.pk = mangadapdb.cleanspaxelprop7.file_pk JOIN mangasampledb.manga_target ON mangasampledb.manga_target.pk = mangadatadb.cube.manga_target_pk JOIN mangasampledb.manga_target_to_nsa ON mangasampledb.manga_target.pk = mangasampledb.manga_target_to_nsa.manga_target_pk JOIN mangasampledb.nsa ON mangasampledb.nsa.pk = mangasampledb.manga_target_to_nsa.nsa_pk JOIN mangadatadb.pipeline_info AS drpalias ON drpalias.pk = mangadatadb.cube.pipeline_info_pk JOIN mangadatadb.pipeline_info AS dapalias ON dapalias.pk = mangadapdb.file.pipeline_info_pk
WHERE CAST(CASE WHEN (mangasampledb.nsa.sersic_mass > 0.0) THEN log(mangasampledb.nsa.sersic_mass) WHEN (mangasampledb.nsa.sersic_mass = 0.0) THEN 0.0 END AS FLOAT) >= 9.5 AND CAST(CASE WHEN (mangasampledb.nsa.sersic_mass > 0.0) THEN log(mangasampledb.nsa.sersic_mass) WHEN (mangasampledb.nsa.sersic_mass = 0.0) THEN 0.0 END AS FLOAT) < 11.0 AND mangasampledb.nsa.sersic_n < 2.0 AND mangadapdb.cleanspaxelprop7.emline_sew_ha_6564 > 6.0 AND drpalias.pk = 32 AND dapalias.pk = 34

### Postgres Results
(2513811 rows)

1st run
Time: 201567.467 ms (03:21.567)

2nd run
Time: 15866.201 ms (00:15.866)

run with HYB10 selection
(1235317 rows) Time: 28622.329 ms (00:28.622)


In [17]:
drpall.describe()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[summary: string, plate: string, ifudsgn: string, plateifu: string, mangaid: string, versdrp2: string, versdrp3: string, verscore: string, versutil: string, versprim: string, platetyp: string, srvymode: string, objra: string, objdec: string, ifuglon: string, ifuglat: string, ifura: string, ifudec: string, ebvgal: string, nexp: string, exptime: string, drp3qual: string, bluesn2: string, redsn2: string, harname: string, frlplug: string, cartid: string, designid: string, cenra: string, cendec: string, airmsmin: string, airmsmed: string, airmsmax: string, seemin: string, seemed: string, seemax: string, transmin: string, transmed: string, transmax: string, mjdmin: string, mjdmed: string, mjdmax: string, gfwhm: string, rfwhm: string, ifwhm: string, zfwhm: string, mngtarg1: string, mngtarg2: string, mngtarg3: string, catidnum: string, plttarg: string, manga_tileid: string, nsa_iauname: string, ifudesignsize: string, ifutargetsize: string, ifudesignwrongsize: string, z: string, zmin: 

In [18]:
drpall.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

11465

In [45]:
from pyspark.sql.functions import log10
with Timer():
    # filter drpall on sersic index and log of stellar mass
    sub = drpall.filter((drpall.nsa_sersic_n < 2.0) & 
                  (log10(drpall.nsa_sersic_mass) >=9.5) & 
                  (log10(drpall.nsa_sersic_mass) < 11.))
    # filter the maps on h-alpha EW
    ew = maps.filter(good_spaxels).filter(maps.emline_sew_ha_6564 > 6)
    # join the tables and select some,count
    tmp = ew.join(sub, 'plateifu')
    tmp.select(tmp.plateifu, tmp.x, tmp.y, tmp.emline_sew_ha_6564, tmp.nsa_sersic_n, log10(tmp.nsa_sersic_mass)).count()

# 0.8-1.2 sec - show(5)
# 2-3 sec - count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1235317
Elapsed time [sec]: 2.3904749280045507

In [20]:
tmp.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1235317

In [21]:
tmp = ew.join(sub, 'plateifu')
tmp.select(tmp.plateifu, tmp.x, tmp.y, tmp.emline_sew_ha_6564, tmp.nsa_sersic_n, log10(tmp.nsa_sersic_mass)).show(5)
tmp.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---+---+------------------+------------+----------------------+
| plateifu|  x|  y|emline_sew_ha_6564|nsa_sersic_n|LOG10(nsa_sersic_mass)|
+---------+---+---+------------------+------------+----------------------+
|8551-3701| 20| 11|29.696453094482422|     1.12056|      9.52915177241679|
|8551-3701| 21| 11| 37.42804718017578|     1.12056|      9.52915177241679|
|8551-3701| 22| 11|38.616214752197266|     1.12056|      9.52915177241679|
|8551-3701| 17| 12| 31.09678077697754|     1.12056|      9.52915177241679|
|8551-3701| 18| 12|29.986642837524414|     1.12056|      9.52915177241679|
+---------+---+---+------------------+------------+----------------------+
only showing top 5 rows

1235317

In [ ]:
#ew.join(sub, 'plateifu').show(1)

### Spark SQL

In [46]:
sql = """select f.plateifu, f.emline_sew_ha_6564, d.nsa_sersic_n, log10(d.nsa_sersic_mass) \
from maps as f join drpall as d on d.plateifu=f.plateifu \
where (f.emline_sew_ha_6564 > 6 and f.binid_binned_spectra != -1 and f.binid_stellar_continua != -1 and \
f.binid_em_line_models != -1 and f.binid_em_line_moments != -1 and f.binid_spectral_indices != -1 and \
d.nsa_sersic_n < 2.0 and log10(d.nsa_sersic_mass) between 9.5 and 11.0)"""

with Timer():
    spark.sql(sql).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1235317
Elapsed time [sec]: 2.072364863997791

## Query 5 
Find all galaxies that have a summed spaxel flux around H-alpha wavelength less than the DAP measured value in spaxels at 1 Ref  


### Raw SQL
with rad as (select f.cube_pk, s.file_pk,s.x, s.y, s.spx_ellcoo_r_re, s.emline_sflux_ha_6564 from mangadapdb.spaxelprop7 as s join mangadapdb.file as f on f.pk=s.file_pk where s.spx_ellcoo_r_re between 0.9 and 1.1 and f.pipeline_info_pk=34 and f.filename ilike '%HYB10%'), red as (select c.pk,n.z, s.sub, s.val from mangadatadb.cube as c join mangasampledb.manga_target as m on m.pk=c.manga_target_pk join mangasampledb.manga_target_to_nsa as nt on m.pk=nt.manga_target_pk join mangasampledb.nsa as n on n.pk=nt.nsa_pk, lateral (select i as sub, rest_wavelength[i] as val from (select generate_subscripts(functions.rest_wavelength(n.z), 1) as i, rest_wavelength from functions.rest_wavelength(n.z)) foo where rest_wavelength[i] between 6563 and 6567) as s where c.pipeline_info_pk=32), flux as (select c.cube_pk, c.x, c.y, c.z, c.spx_ellcoo_r_re, sum(c.unnest) as sflux, c.emline_sflux_ha_6564 from (select b.*, red.z from (select s.cube_pk, s.x, s.y, unnest(s.flux), generate_subscripts(s.flux,1) as fsub, rad.spx_ellcoo_r_re, rad.emline_sflux_ha_6564 from mangadatadb.spaxel as s, rad where s.cube_pk=rad.cube_pk and s.x=rad.x and s.y=rad.y) as b, red where b.fsub = red.sub and b.cube_pk=red.pk) as c group by c.cube_pk, c.x, c.y, c.z, c.spx_ellcoo_r_re, c.emline_sflux_ha_6564) select flux.cube_pk, c.plate,i.name, flux.z from flux join mangadatadb.cube as c on c.pk=flux.cube_pk join mangadatadb.ifudesign as i on i.pk=c.ifudesign_pk where flux.emline_sflux_ha_6564 > flux.sflux;

### Postgres Results
1st run 
(390604 rows) - duplicate rows

Time: 19425544.471 ms (05:23:45.544)

2nd run
(3117 rows) - unique rows, with extra group by

Time: 17431038.283 ms (04:50:31.038)

In [16]:
good_spaxels = ((maps.binid_binned_spectra != -1) & 
                   (maps.binid_stellar_continua != -1) & 
                   (maps.binid_em_line_models != -1) & 
                   (maps.binid_em_line_moments != -1) & 
                   (maps.binid_spectral_indices != -1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
from pyspark.sql.functions import monotonically_increasing_id, expr
rad = maps.filter(good_spaxels).filter((maps.spx_ellcoo_r_re >= 0.9) & 
                   (maps.spx_ellcoo_r_re < 1.1))

cubedrp = cubes.join(drpall, (cubes['plateid'] == drpall.plate) & (cubes['ifudsgn']==drpall.ifudsgn)).select(cubes["flux"], cubes["wave"], cubes["_src_x"], cubes["_src_y"],drpall["plateifu"],drpall["nsa_z"])
wave = cubedrp.withColumn("wave_idx", monotonically_increasing_id() % 4563).withColumn('restw', cubedrp['wave']/(1+cubedrp['nsa_z']))
wave.filter((wave.restw > 6563) & (wave.restw < 6567))

wave.groupby(wave.plateifu, wave._src_x, wave._src_y).sum().show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+------+------------------+--------------------+-----------+-----------+------------------+-------------+--------------------+
|  plateifu|_src_x|_src_y|         sum(flux)|           sum(wave)|sum(_src_x)|sum(_src_y)|        sum(nsa_z)|sum(wave_idx)|          sum(restw)|
+----------+------+------+------------------+--------------------+-----------+-----------+------------------+-------------+--------------------+
|7977-12703|    39|    58|1.9382832507751573|2.9244602021747213E7|     177957|     264654|337.03550010002357|     10408203|2.7233092295455687E7|
|7977-12703|    42|    35|1036.5087170015904|2.9244602021747213E7|     191646|     159705|337.03550010002357|     10408203|2.7233092295455687E7|
|7977-12703|    43|    41| 520.9715455719561|2.9244602021747213E7|     196209|     187083|337.03550010002357|     10408203|2.7233092295455687E7|
|7977-12703|    47|    53| 8.723400353684156|2.9244602021747213E7|     214461|     241839|337.03550010002357|     10408203|2.72330

In [50]:
drpall.first().plateifu

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'8935-12701'

In [59]:
from pyspark.sql.functions import concat, col, lit, expr, min, max
#ctemp = cubes.withColumn('plateifu', concat(col("plateid"), lit("-"), col("ifudsgn")))
#ctemp.first().plateifu

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#ctemp.join(drpall, 'plateifu').first()

In [63]:
cubes = cubes.withColumn('wave_idx', expr("wave % 4563"))
cubes.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+------------------+------+------+--------------------+------------------+----+------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+---+---+---+---+-------+-------+--------+--------+------+----------+------------------+
|                RA|               DEC|_SRC_X|_SRC_Y|                FLUX|              IVAR|MASK|              DISP|           PREDISP|              WAVE|           SPECRES|         SPECRESD|        PRESPECRES|      PRESPECRESD|  G|  R|  I|  Z|PLATEID|IFUDSGN|DESIGNID| MANGAID|MJDRED|  DATE-OBS|          wave_idx|
+------------------+------------------+------+------+--------------------+------------------+----+------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+---+---+---+---+-------+-------+--------+--------+------+----------+------------------+
|333.20683578159424|13.333565395125403|     2|   

In [68]:
cubes.agg({"wave": "max"}).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(max(wave)=10353.805444147549)]

### Spark SQL

In [7]:
radsql = """select f.plateifu, f.emline_sflux_ha_6564, f.x, f.y, f.spx_ellcoo_r_re from maps as f \
where ((f.spx_ellcoo_r_re between 0.9 and 1.1) and f.binid_binned_spectra != -1 and f.binid_stellar_continua != -1 and \
f.binid_em_line_models != -1 and f.binid_em_line_moments != -1 and f.binid_spectral_indices != -1)"""

redsql = """select i.plateifu, i.wave, i.rw, i.wave_idx from (select d.plateifu, c.wave, c.wave / (1+d.nsa_z) as rw, \
monotonically_increasing_id() % 4563 as wave_idx from cubes as c join drpall as d on d.plate=c.plateid \
and d.ifudsgn=c.ifudsgn) as i where i.rw between 6563 and 6567"""

fluxsql = """select r.plateifu, sum(c.flux) as sflux, c._src_x as x, c._src_y as y from ({red}) as r \
join cubes as c on c.wave=r.wave group by r.plateifu, c._src_x, c._src_y""".format(red=redsql)

flux = """select f.plateifu, f.x, f.y, f.sflux, r.emline_sflux_ha_6564 from ({flux}) as f, \
({rad}) as r where r.plateifu=f.plateifu and r.x=f.x and r.y=f.y \
and r.emline_sflux_ha_6564 > f.sflux""".format(flux=fluxsql, rad=radsql)

final = """select g.plateifu, count(g.*) from ({ff}) as g group by g.plateifu""".format(ff=flux)
with Timer():
    spark.sql(final).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '500' from https://172.23.25.61:30443/gateway/default/livy/v1/sessions/11/statements/6 with error payload: "java.lang.IllegalStateException: RPC channel is closed."


In [96]:
spark.sql('''describe cubes''').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+-------+
|   col_name|data_type|comment|
+-----------+---------+-------+
|         RA|   double|   null|
|        DEC|   double|   null|
|     _SRC_X|   bigint|   null|
|     _SRC_Y|   bigint|   null|
|       FLUX|   double|   null|
|       IVAR|   double|   null|
|       MASK|   bigint|   null|
|       DISP|   double|   null|
|    PREDISP|   double|   null|
|       WAVE|   double|   null|
|    SPECRES|   double|   null|
|   SPECRESD|   double|   null|
| PRESPECRES|   double|   null|
|PRESPECRESD|   double|   null|
|          G|   double|   null|
|          R|   double|   null|
|          I|   double|   null|
|          Z|   double|   null|
|    PLATEID|   bigint|   null|
|    IFUDSGN|   bigint|   null|
+-----------+---------+-------+
only showing top 20 rows

In [9]:
a='5'
b='{a}'.format(a=a)
print(b)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5